In [3]:
import os
os.environ["OPENAI_API_KEY"] = 'your_api_key'

In [35]:
from langchain.vectorstores import Chroma, ElasticKnnSearch
from langchain.embeddings import OpenAIEmbeddings, cohere as ce
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, CharacterTextSplitter
)
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, ConversationChain
from langchain.document_loaders import (
    TextLoader, DirectoryLoader, PyPDFLoader
)
from langchain.prompts import (
    PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
)
from langchain.chat_models import ChatOpenAI
from langchain.memory import (
    ChatMessageHistory, ConversationBufferMemory, ConversationBufferWindowMemory
)


In [36]:
# PyPDFLoader를 사용하여 'investment.pdf' 파일을 로드합니다.
loader = PyPDFLoader('/content/drive/MyDrive/investment.pdf')
# 로더를 사용하여 PDF 파일의 내용을 documents 변수에 로드합니다.
documents = loader.load()
# 'CharacterTextSplitter'를 사용하여 텍스트를 크기 1000의 청크로 분할합니다. 청크 간의 겹침은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
# 분할자를 사용하여 'documents'를 청크로 분할하고 결과를 texts 변수에 저장합니다.
texts = text_splitter.split_documents(documents)

In [7]:
#벡터 데이터베이스가 저장될 디렉토리의 이름을 변수에 저장합니다.
persist_directory = 'db'
# 텍스트 문서의 임베딩을 생성하는데 사용할 모델을 변수에 할당합니다.
embedding = OpenAIEmbeddings()
# Chroma 클래스의 'from_documents' 메서드를 호출하여 문서의 임베딩을 생성하고 벡터데이터 베이스를 초기화합니다.
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory)

In [37]:
# persist 메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 디스크에 저장합니다.
# 이렇게 하면 나중에 데이터베이스를 다시 로드하여 사용할 수 있습니다.
vectordb.persist()
# 메모리에서 데이터베이스를 제거합니다.
vectordb = None
# Chroma 클래스를 인스턴스화하여 vectordb 변수에 할당합니다. 문서의 임베딩을 저장하고 검색하는데 사용됩니다.
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding)

In [9]:
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색가능한 형태로 변환합니다.
retriever = vectordb.as_retriever()
# 질문과 관련된 문서를 검색합니다.(기본 k값은 4로 설정되어있어, 관련도가 높은 4개의 문서를 검색합니다.)
docs = retriever.get_relevant_documents("디폴트 옵션이 뭐야?")
# 각 문서의 페이지 번호를 가져와 출력합니다.
for doc in docs:
    print(doc.metadata["page"])

0
1
24
71


In [19]:
# 프롬프트를 'prompt_template'변수에 저장합니다.
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
you are competent financial advisor.

{context}

Question: {question}

Don't answer with short answers, answer with complete sentences.
Answer in Korean:"""
# PROMPT에 'context'와 'question'을 입력 변수로하는 프롬프트 템플릿을 저장합니다.
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
# 'chain_type_kwargs'변수에 딕셔너리를 할당합니다.
chain_type_kwargs = {"prompt": PROMPT}

In [20]:
# 'as_retriever'메서드를 호출하여 'vectordb'에서 생성된 벡터 데이터베이스를 검색 가능한 형태로 변환합니다.
# 유사도가 높은 상위 3개의 문서만 반환하도록 지정합니다.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
# 'gpt-3.5-turbo-16k'모델을 사용합니다.
# temperature값이 낮은수록 더 일관성 있는 문장이 생성됩니다.
llm = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo-16k')

# 체인타입을 결정합니다.
qa_chain = RetrievalQA.from_chain_type(
    # 언어모델을 할당합니다.
    llm=llm,
    # 체인의 유형을 지정합니다.
    chain_type="stuff",
    retriever=retriever,
    # 응답과 함께 소스 문서를 반환합니다.
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

# 언어 모델의 응답을 처리하는 함수를 정의합니다.
def process_llm_response(llm_response):
    # 언어 모델의 응답 결과를 출력합니다.
    print(llm_response['result'])
    # 언어 모델 응답의 소스 문서 목록을 반환합니다.
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata["page"])

In [33]:
query = "23년 하반기는 경제가 좋아질까?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

23년 하반기에 경제가 좋아질지에 대해서는 확실한 답을 내리기 어렵습니다. 현재 경기 침체 가능성이 높아지고 있으며, 인플레이션과 미 연준의 통화정책 변화 등 여러 불확실성 요인이 존재합니다. 하지만 경기 침체의 폭과 기간이 과거와 비교해 상대적으로 낮을 것으로 전망되고 있으며, 미 연준의 긴축 강도가 완화될 가능성도 있습니다. 따라서 하반기부터는 경제의 회복 기대가 높아질 수 있습니다. 그러나 이는 예측이며, 실제 경제 상황에 따라 변동할 수 있습니다.


Sources:
11
20
30


In [34]:
query = "23년 상반기는 어땠어?" + "이전 질문은 " + query + "이니깐 고려해서 답해줘"
llm_response = qa_chain(query)
process_llm_response(llm_response)

23년 상반기는 경제 여건이 부진한 상황이었습니다. 작년 4분기와 이번 해 1분기에는 경제성장률이 각각 -0.4%와 +0.3%로 역성장 및 미미한 성장을 기록했으며, 세계은행은 미국 경제성장률이 +0.5%로 예상했습니다. 또한, 글로벌 인플레이션과 금리도 높은 수준을 유지하고 있었습니다. 이로 인해 투자자들의 심리가 위축되었고, 주식시장에서는 손실을 보고 있는 사람들이 추가 투자를 꺼리는 상황이었습니다. 그러나 주요 주식시장은 상승세를 보였으며, 삼성전자와 SK하이닉스 등 일부 기업의 주가는 메모리 반도체 가격의 상승 전환 전망에 힘입어 상승했습니다. 따라서 23년 상반기는 경제가 좋지 않은 상황이었지만, 주식시장에서는 일부 기업의 주가가 상승하는 모습이 보였습니다.


Sources:
11
30
27


In [29]:
query = "니가 가지고 있는 데이터를 바탕으로 추천질문 하나 만들어줘"
llm_response = qa_chain(query)
process_llm_response(llm_response)

은퇴 후의 재정적인 안정을 위해 어떤 투자 전략을 추천하시겠습니까?


Sources:
37
10
21


In [30]:
query = "은퇴 후의 재정적인 안정을 위해 어떤 투자 전략을 추천하시겠습니까?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

은퇴 후의 재정적인 안정을 위해서는 변동성이 낮고 안정적인 수익을 창출할 수 있는 포트폴리오를 구성하는 것이 중요합니다. 주식과 채권과 같이 서로 다른 특징을 가진 자산을 적절히 배분하여 꾸준한 수익을 창출할 수 있는 전략을 추천합니다. 또한, 투자자의 기대 수익률과 시장 상황에 따라 포트폴리오의 구성을 조정하고, 적절한 자산 배분을 통해 안정적인 수익을 추구할 수 있습니다. 마켓 타이밍이나 종목 선정보다는 적절한 자산 배분이 투자 수익을 결정하는 가장 중요한 요소라고 생각합니다. 따라서, 현재의 투자 포트폴리오가 안정적인 수익을 창출할 수 있도록 구성되어 있는지 다시 살펴보는 것이 필요합니다.


Sources:
4
38
19
